In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
import os
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from collections import deque, namedtuple
import copy
from swig import swig
from tqdm import tqdm

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# Enable autograd anomaly detection
torch.autograd.set_detect_anomaly(True)

2.4.1+cu121
12.1
True


In [2]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [10]:
class Actor(nn.Module):
    def __init__(self, input_shape, action_size):
        super(Actor, self).__init__()
        self.conv1 = nn.Conv2d(input_shape[2], 16, kernel_size=4, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1)

        conv_out_size = self._get_conv_output(input_shape)

        self.fc1 = nn.Linear(conv_out_size, 256)
        self.fc2 = nn.Linear(256, action_size)
        
        # Weight initialization
        nn.init.kaiming_uniform_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv2.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv3.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity='relu')
        nn.init.uniform_(self.fc2.weight, -3e-3, 3e-3)  # Smaller range for final layer

    def _get_conv_output(self, shape):
        input = torch.rand(1, *shape)
        input = input.permute(0, 3, 1, 2)
        output = self.conv3(self.conv2(self.conv1(input)))
        return int(np.prod(output.size()))

    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(0)
        if x.size(1) != 3:
            x = x.permute(0, 3, 1, 2)
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        action = torch.tanh(self.fc2(x)) * 1.0  # Scale output to match action space range
        return action


class Critic(nn.Module):
    def __init__(self, input_shape, action_size):
        super(Critic, self).__init__()
        self.conv1 = nn.Conv2d(input_shape[2], 16, kernel_size=4, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1)

        conv_out_size = self._get_conv_output(input_shape)

        self.fc1 = nn.Linear(conv_out_size + action_size, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 1)

        # Weight initialization
        nn.init.kaiming_uniform_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv2.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv3.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.fc2.weight, nonlinearity='relu')
        nn.init.uniform_(self.fc3.weight, -3e-3, 3e-3)  # Smaller range for final layer

    def _get_conv_output(self, shape):
        input = torch.rand(1, *shape)
        input = input.permute(0, 3, 1, 2)
        output = self.conv3(self.conv2(self.conv1(input)))
        return int(np.prod(output.size()))

    def forward(self, state, action):
        if state.dim() == 3:
            state = state.unsqueeze(0)
        if state.size(1) != 3:
            state = state.permute(0, 3, 1, 2)
        
        x = F.relu(self.conv1(state))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = torch.cat([x, action], dim=1)  # Concatenate state and action
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        Q_value = self.fc3(x)
        return Q_value

In [11]:
import gymnasium as gym
env = gym.make("CarRacing-v3", render_mode="rgb_array", lap_complete_percent=0.95, domain_randomize=False, continuous=True)
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]
action_space = env.action_space
print("State shape: ", state_shape)
print("state size: ", state_size)
print("action size: ", action_size)
print("action space:", action_space)

State shape:  (96, 96, 3)
state size:  96
action size:  3
action space: Box([-1.  0.  0.], 1.0, (3,), float32)


In [12]:
critic_learning_rate = 1e-3
minibatch_size = 128
discount_factor = 0.99
replay_buffer_size = int(1e5)
tau = 1e-3 

In [5]:
class ReplayMemory:
    def __init__(self, capacity, state_shape, action_dim, device):
        self.capacity = capacity
        self.device = device
        self.memory = deque(maxlen=capacity)
        self.state_shape = state_shape
        self.action_dim = action_dim

    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        
        # Pre-allocate numpy arrays
        states = np.zeros((batch_size, *self.state_shape), dtype=np.float32)
        actions = np.zeros((batch_size, self.action_dim), dtype=np.float32)
        rewards = np.zeros((batch_size, 1), dtype=np.float32)
        next_states = np.zeros((batch_size, *self.state_shape), dtype=np.float32)
        dones = np.zeros((batch_size, 1), dtype=np.float32)

        # Fill the numpy arrays
        for i, (state, action, reward, next_state, done) in enumerate(batch):
            states[i] = state
            actions[i] = action
            rewards[i] = reward
            next_states[i] = next_state
            dones[i] = done

        # Convert numpy arrays to tensors
        return (
            torch.from_numpy(states).pin_memory().to(self.device,non_blocking=True),
            torch.from_numpy(actions).to(self.device),
            torch.from_numpy(rewards).to(self.device),
            torch.from_numpy(next_states).to(self.device),
            torch.from_numpy(dones).to(self.device)
        )

    def __len__(self):
        return len(self.memory)

In [13]:
class Agent:
    def __init__(self, state_shape, action_size, actor_lr=1e-4, critic_lr=1e-3, replay_buffer_size=10000, minibatch_size=128, gamma=0.99, tau=0.005):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.state_shape = state_shape
        self.action_size = action_size
        self.gamma = gamma
        self.tau = tau

        # Actor network
        self.actor = Actor(state_shape, action_size).to(self.device)
        self.target_actor = copy.deepcopy(self.actor)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=actor_lr)

        # Critic network
        self.critic = Critic(state_shape, action_size).to(self.device)
        self.target_critic = copy.deepcopy(self.critic)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=critic_lr)

        # Replay buffer
        self.memory = ReplayMemory(replay_buffer_size, state_shape, action_size, self.device)
        self.minibatch_size = minibatch_size

    def act(self, state, noise_scale=0.1):
        """Get action using the actor network and add noise for exploration."""
        self.actor.eval()
        with torch.no_grad():
            state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
            action = self.actor(state_tensor).cpu().data.numpy().squeeze()
        self.actor.train()

        # Scale action outputs to the environment's action space
        action = (action + 1) / 2  # Map from [-1, 1] to [0, 1]
        action[0] = action[0] * 2 - 1  # Scale the first action to [-1, 1]

        # Add noise for exploration, scaled per action dimension
        noise = noise_scale * np.random.randn(self.action_size)
        action = action + noise

        # Clip each action dimension according to the specific action space limits
        action = np.clip(action, env.action_space.low, env.action_space.high)
        return action


    def step(self, state, action, reward, next_state, done):
        """Save experience to replay memory and learn if enough samples."""
        self.memory.push(state, action, reward, next_state, done)
        if len(self.memory) > self.minibatch_size:
            experiences = self.memory.sample(self.minibatch_size)
            self.learn(experiences)

    def learn(self, experiences):
        """Update actor and critic networks from sampled experiences."""
        states, actions, rewards, next_states, dones = experiences

        # Critic update: Q-value estimation
        with torch.no_grad():
            next_actions = self.target_actor(next_states)
            Q_targets_next = self.target_critic(next_states, next_actions)
            Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))

        Q_expected = self.critic(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Actor update: Maximize Q-value predicted by the critic
        actions_pred = self.actor(states)
        actor_loss = -self.critic(states, actions_pred).mean()

        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Soft update of target networks
        self.soft_update(self.critic, self.target_critic, self.tau)
        self.soft_update(self.actor, self.target_actor, self.tau)

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters."""
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)


In [14]:
agent = Agent(state_shape, action_size)

In [15]:
class OUNoise:
    """Ornstein-Uhlenbeck process for temporally correlated noise."""
    def __init__(self, size, mu=0.0, theta=0.15, sigma=0.2):
        self.size = size
        self.mu = mu
        self.theta = theta
        self.sigma = sigma
        self.state = np.ones(self.size) * self.mu
        self.reset()

    def reset(self):
        self.state = np.ones(self.size) * self.mu

    def sample(self):
        dx = self.theta * (self.mu - self.state) + self.sigma * np.random.randn(self.size)
        self.state += dx
        return self.state

def train_agent(agent, env, number_episodes=5000, max_timesteps_per_episode=500, target_score=50.0):
    scores_window = deque(maxlen=100)
    
    noise = OUNoise(agent.action_size, sigma=0.2) 

    for episode in tqdm(range(1, number_episodes + 1), desc="Training Progress"):
        state, _ = env.reset()
        score = 0
        noise.reset()  # Reset noise at the start of each episode

        for t in range(max_timesteps_per_episode):
            action = agent.act(state)  
            action += noise.sample()  # Add Ornstein-Uhlenbeck noise for exploration
            action = np.clip(action, -1, 1)  # Clip action to be within the valid range

            next_state, reward, done, _, _ = env.step(action)
            
            agent.step(state, action, reward, next_state, done)  # Store experience and train if needed
            
            state = next_state
            score += reward

            if done:
                break
        
        scores_window.append(score)

        # Print updates and save models every 100 episodes
        if episode % 100 == 0:
            avg_score = np.mean(scores_window)
            print(f'\rEpisode {episode}\tAverage Score: {avg_score:.2f}')
            torch.save(agent.actor.state_dict(), f'checkpoint_actor.pth')
            torch.save(agent.critic.state_dict(), f'checkpoint_critic.pth')

        # Check if the environment is solved
        if np.mean(scores_window) >= target_score:
            print(f'\nEnvironment solved in {episode - 100:d} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
            torch.save(agent.actor.state_dict(), 'checkpoint_actor_final.pth')
            torch.save(agent.critic.state_dict(), 'checkpoint_critic_final.pth')
            return True

    # Save the final models if the environment is not solved
    print(f'\nEnvironment not solved. Final Average Score: {np.mean(scores_window):.2f}')
    torch.save(agent.actor.state_dict(), 'checkpoint_actor_final_us.pth')
    torch.save(agent.critic.state_dict(), 'checkpoint_critic_final_us.pth')
    return False

# Usage:
train_agent(agent, env)

Training Progress:   2%|▏         | 100/5000 [51:12<41:30:35, 30.50s/it]

Episode 100	Average Score: -25.91


Training Progress:   4%|▍         | 200/5000 [1:41:55<40:41:41, 30.52s/it]

Episode 200	Average Score: -25.25


Training Progress:   6%|▌         | 300/5000 [2:32:38<39:50:10, 30.51s/it]

...

Episode 1600	Average Score: -25.09


Training Progress:  34%|███▍      | 1700/5000 [14:20:52<27:41:52, 30.22s/it]

Episode 1700	Average Score: -25.32


Training Progress:  36%|███▌      | 1778/5000 [15:00:27<27:11:45, 30.39s/it]


KeyboardInterrupt: 

In [16]:
import torch
import imageio
import gymnasium as gym
import base64
import glob
import io
from IPython.display import HTML, display

# Check for GPU and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the trained model
def load_actor_model(actor_path, input_shape, action_size):
    actor = Actor(input_shape=input_shape, action_size=action_size).to(device)
    actor.load_state_dict(torch.load(actor_path, map_location=device))
    actor.eval()  # Set the model to evaluation mode
    return actor

# Show video of the trained model's performance with a limited number of steps
def show_video_of_model(actor, env_name, input_shape, video_filename='video.mp4', max_steps=1000):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False

    # Open a writer to save frames on-the-fly to reduce memory usage
    with imageio.get_writer(video_filename, fps=30) as video_writer:
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
        step_count = 0  # Counter for the number of steps

        while not done and step_count < max_steps:
            frame = env.render()
            video_writer.append_data(frame)  # Save frame directly to disk

            with torch.no_grad():  # Disable gradient computation for inference
                action = actor(state).squeeze(0).cpu().numpy()  # Move to CPU for env.step()

            # Step in the environment and process the new state
            state, reward, done, _, _ = env.step(action)
            state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)

            step_count += 1  # Increment the step counter

    env.close()

# Load the actor model and show the video
input_shape = (96, 96, 3)
action_size = 3
actor = load_actor_model('checkpoint_actor.pth', input_shape, action_size)

# Limit to a smaller number of steps for quicker testing
show_video_of_model(actor, 'CarRacing-v3', input_shape, max_steps=500)

# Display the video
def show_video(video_filename='video.mp4'):
    try:
        video = io.open(video_filename, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data=f'''
        <video width="640" height="480" controls>
            <source src="data:video/mp4;base64,{encoded.decode('ascii')}" type="video/mp4">
        </video>
        '''))
    except FileNotFoundError:
        print("Could not find video file.")

show_video()


/tmp/ipykernel_4834/3542375078.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  actor.load_state_dict(torch.load(actor_path, map_location=device))
IMAGEIO FFMPEG_WRITER 

Using device: cuda


[swscaler @ 0x5e4de00] Warning: data is not aligned! This can lead to a speed loss
